In [245]:
import pandas as pd
import openai
import constants
import tiktoken

openai.api_key = constants.OPENAI_KEY

In [246]:
# Prevent truncation of text
pd.set_option('display.max_colwidth', None)
# Show more rows
pd.set_option('display.max_rows', 150)

In [247]:
ARBITARY_MAX_TOKENS = 3500
ARBITARY_2_QUESTION_TOKENS = 200

In [248]:
# If the file articles_questions.csv exists, load it, otherwise load the articles.csv file
try:
    df_sample = pd.read_csv("articles_questions.csv")
except:
    df = pd.read_csv("articles.csv")
    # Add columns question_1 and question_2 to the dataframe
    df["question_1"] = pd.NA
    df["question_2"] = pd.NA
    # Take 2000 random rows from the dataframe
    df_sample = df.sample(n=2000, random_state=1)

df_sample.head()

,title,article,published,article_id,question_1,question_2
0,Krigsoro bland finska barn: ”Mormor bor vid gränsen”,"Även om det inte finns något direkt ryskt hot mot Finland i nuläget så gränsar de båda länderna till varandra. Något som har skapat en oro för att konflikten i Ukraina ska sprida sig till landet, inte minst bland barn och unga. BBC har besökt en skola i finska Uleåborg, 25 mil från den ryska gränsen, för att prata med eleverna om deras syn på kriget. – Min mormor bor i Kuhmo som ligger väldigt nära gränsen. Jag är orolig för att hon inte ska överleva om Ryssland attackerar, berättar 12-åriga Riikka.15-åriga Olli säger att hans orosnivå höjdes från en tvåa, på en tiogradig skala, till en åtta när den ryska invasionen i Ukraina inleddes. – Men nu börjar jag vänja mig vid kriget, jag börjar sakta bli avtrubbad, säger han.",2022-03-30T08:32:25Z,34OlEd,Vad oroar eleverna på skolan i Uleåborg när det gäller konflikten i Ukraina?,Hur har 15-åriga Ollis orosnivå förändrats under konflikten i Ukraina?
1,Microsoft har upptäckt 237 cyberattacker mot Ukraina,"Den amerikanska teknikjätten Microsoft har upptäckt 237 cyberattacker mot Ukraina sedan den ryska invasionen inleddes. Det skriver bolaget i en rapport, enligt TT.Attackerna ska ha kommit från minst sex olika länder som står nära Ryssland, och det rör sig främst om försök att störa myndigheters och företags verksamhet. Desinformation har också skickats ut – exempelvis via mejl – direkt riktad mot civila. Enligt Microsoft tycks attackerna vara koordinerade med den militära verksamheten, och teknikjätten tror att de kommer att de kommer att eskalera i takt även fortsättningsvis.",2022-04-28T03:16:07Z,nW2yVo,Hur många cyberattacker har Microsoft upptäckt mot Ukraina sedan den ryska invasionen inleddes?,Vilka länder har enligt Microsoft utfört cyberattacker mot Ukraina?
2,Ökad risk för olyckor när svenskar preppar för kris,"När allt fler svenskar preppar till följd av kriget i Ukraina kan också fler komma att ha brandfarliga varor, som gas och bensin, hemma, rapporterar SVT Västmanland. För den som börjat bunkra är det viktigt att ta reda på vad som gäller, säger brandingenjören Fredrik Eriksson på Räddningstjänsten Mälardalen. Exempelvis får personer som bor i flerfamiljshus inte förvara gas och bensin i förrådet, på vinden eller i källaren.Efterfrågan hos svenska försvarsföretag ökade markant redan innan krigsutbrottet, rapporterar TT. På företaget Snigeldesign, som utvecklar skyddsvästar och ryggsäckar för att bära utrustning, tredubblades omsättningen i fjol till följd av geopolitisk oro.– Det finns helt plötsligt ett annat intresse att tidigarelägga och utöka redan befintliga kontrakt, säger vd:n Carl Jonasson.",2022-03-20T07:53:04Z,MLb3pK,Varför kan fler svenskar ha brandfarliga varor hemma till följd av kriget i Ukraina?,Vilket företag har tredubblat sin omsättning till följd av geopolitisk oro?
3,Biden: Redo för samtal om Putin verkligen vill ha fred,"Den amerikanske presidenten Joe Biden är redo att samtala med Rysslands Vladimir Putin, om den ryske presidenten visar ett genuint intresse av att avsluta kriget i Ukraina. Det sa Biden i samband med ett besök av Frankrikes president Emanuel Macron, skriver Financial Times.Biden sa dock att han inte kommer att träffa Putin utan att först ha konsulterat sina allierade inom Nato. Han sa också att han i nuläget inte ser några tecken på att Putin är villig att avsluta kriget.På en gemensam presskonferens underströk också Biden och Macron att deras länder tillsammans med resten av Nato och EU står enade mot Ryssland, skriver AFP. Macron underströk att Ukraina aldrig kommer att pressas att gå med på en fredsuppgörelse som inte är acceptabel ur ukrainskt perspektiv.",2022-12-02T05:04:31Z,P4gGA5,Vad kräver Joe Biden av Vladimir Putin för att han ska vara redo att samtala med honom?,Vad sa Joe Biden om Putins vilja att avsluta kriget i Ukraina?
4,En ny sorts luftstrid: Så slåss drönare mot drönare över Ukraina,"Glöm pistoler och 

In [249]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def calculate_tokens_conversation(messages):
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

def calculate_tokens_string(text):
    return len(encoding.encode(text))

In [250]:
def get_questions(articles: dict):
    instructions = f"""Detta system kommer att generera frågor om de artiklar som användaren matar in. Systemet måste generera exakt två frågor per artikel. Frågorna bör kunna besvaras med information från artikeln.
    
    Frågorna ska formuleras på ett sådant sätt så att man inte ska behöva läst artikeln för att förstå frågorna. Frågorna kan vara mycket specifika eller lite bredare. Referera inte till artikeln eller andra källor i frågorna, frågorna ska alltså vara självständiga.
    
    Returnera alla frågorna i ordning.
    
    Svaret ska vara formatterat som en python dictionary där nyckeln är artikel id och värdet är en lista med artikels två frågor. Svaret ska gå att laddas in som en dictionary med eval().
    """
    
    articles_string = ""
    for article_id, article_text in articles.items():
        articles_string += "Artikel " + article_id + ":\n"
        articles_string += article_text + "\n\n"

    prompt = [{"role": "system", "content": instructions}]
    prompt.append({"role": "user", "content": articles_string})

    max_token_answer = len(articles) * ARBITARY_2_QUESTION_TOKENS

    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = prompt,
        temperature = 0.6,
        max_tokens = max_token_answer,
        stream = False,
    )

    answer_string = response["choices"][0]["message"]["content"]
    answer = eval(answer_string)
    return answer

In [251]:
def handle_batch_of_articles(df_local: pd.DataFrame, articles: dict):
    questions = get_questions(articles)

    for article_id, questions_list in questions.items():
        df_local.loc[df_local["article_id"] == article_id, "question_1"] = questions_list[0]
        df_local.loc[df_local["article_id"] == article_id, "question_2"] = questions_list[1]

    df_local.to_csv("articles_questions.csv", index=False)


In [252]:
def populate_questions(df_local: pd.DataFrame):
    # For articles where the questions are not already filled in, generate questions. Use calculate_tokens to make sure that the number of tokens is not too high. For every article added 200 tokens should also be reserved for the questions. The total number of tokens should not exceed 4000.
    total_data_points = len(df_local)
    current_data_point = 0

    added_articles = {}
    tokens_articles = 0
    for _, row in df_local.iterrows():
        current_data_point += 1
        if pd.isna(row["question_1"]) or pd.isna(row["question_2"]):
            tokens_article = calculate_tokens_string(row["article"])

            if tokens_article + tokens_articles + len(added_articles) * ARBITARY_2_QUESTION_TOKENS < ARBITARY_MAX_TOKENS:
                added_articles[row["article_id"]] = row["article"]
                tokens_articles += tokens_article
            else:
                handle_batch_of_articles(df_local, added_articles)
                added_articles = {row["article_id"]: row["article"]}
                tokens_articles = tokens_article
                print(f"Processed {current_data_point} of {total_data_points} articles. ({current_data_point / total_data_points * 100:.2f} %)")

    if(len(added_articles) > 0):
        handle_batch_of_articles(df_local, added_articles)

In [253]:
populate_questions(df_sample)

In [255]:
df_sample.tail()

,title,article,published,article_id,question_1,question_2
1995,Polska pandemisjukhus kan bli avgörande för flyktingar,"Sjukhus i Ukrainas grannländer gör sig redo för att ta emot människor som flyr kriget – och Polen har redan tagit emot sina första ukrainska flyktingar i behov av vård. Det skriver Politico.I östra Polen handlar det till största del om barn, födande kvinnor, äldre och människor med kroniska sjukdomar vars tillstånd har förvärrats. Patienterna tas emot av sjukhus som dragit lärdomar från pandemin och som byggt upp en stark infrastruktur. Alla storstäder har under pandemin upprättat fältsjukhus, som står kvar än i dag, berättar chefen för ett polskt sjukvårdsförbund.– Så infrastrukturen är egentligen inte en stor utmaning. Enligt Politico kan fältsjukhusen komma att bli avgörande om Polen tar emot en stor mängd flyktingar i behov av traumavård.",2022-03-03T18:53:10Z,oWzebW,Vilka typer av människor gör sig redo att fly från kriget i Ukraina och ta emot vård i sjukhus i grannländerna?,Vad har sjukhusen i Polen gjort för att förbereda sig för att ta emot flyktingar i behov av vård?
1996,”Flyktingkatastrof av en skala Europa inte upplevt sedan andra världskriget”,"Rysslands invasion av Ukraina innebär ”en flyktingkatastrof av en skala som Europa inte upplevt sedan andra världskriget”, enligt Filippo Grandi, FN:s flyktingkommissarie. I en intervju med Dagens Nyheter i samband med ett Sverigebesök säger han att han håller med statsminister Magdalena Andersson (S), som för ett par veckor sedan sa att fler länder måste ta ansvar för flyktingarna jämfört med 2015. – Då var det i hög grad Sverige och Tyskland som ställde upp som mottagare, säger Grandi till DN. Han säger också att det inte går att begära att grannlandet Polen tar emot två till tre miljoner flyktingar, utan att ansvaret måste fördelas.",2022-03-11T17:47:47Z,v5Pe3X,Vilken skala har flyktingkatastrofen som Filippo Grandi beskriver i artikeln?,Vilket land tog emot flest flyktingar under 2015 enligt Filippo Grandi?
1997,Zelenskyj varnar för minor: ”Terroristernas dödsfällor”,"Ukrainas president Volodymyr Zelenskyj varnar i sitt nattliga tal för att kvarlämnade ryska minor kommer att ta flera år att röja undan.– Terroristerna lämnar medvetet kvar så många dödsfällor som möjligt, säger Zelenskyj.Enligt presidenten är mer än 170 000 kvadratkilometer av Ukraina ”farligt territorium”. Nyligen uppgav Ukrainas inrikesdepartement att omkring en tredjedel av landet är farligt på grund av explosiva objekt, rapporterar Radio Free Europe.Zelenskyj menar att kvarlämnandet av minor kommer bli ett av de viktigaste åtalen mot Ryssland efter kriget. Minorna är ännu värre än robotarna, enligt honom, eftersom robotarna åtminstone går att försvara sig mot.",2022-12-09T07:41:03Z,wAdO05,Varför varnar Ukrainas president för farliga minor?,Hur stor del av Ukraina är farligt på grund av explosiva objekt enligt Radio Free Europe?
1998,Ryssland kapar elförsörjning till Finland,"Ryssland kapar försörjningen av el till Finland på lördagen, uppger RAO Nordic, ett dotterbolag till den ryska statliga leverantören Inter RAO, enligt AFP.I ett pressmeddelande uppger RAO Nordic att det beror på att man inte kan betala för elimporter från Ryssland.”Situationen är exceptionell och sker för första gången i vår tjugoåriga handelshistoria”, skriver bolaget.Fingrid, som förvaltar stamnätet i Finland, skriver i ett pressmeddelande att beskedet – som berör runt tio procent av den totala förbrukningen – inte hotar den finländska elförsörjningen.– Den import som saknas kan ersättas genom import av mer el från Sverige och delvis också genom finska produktionen, säger Reima Päivinen, ansvarig direktör för driftverksamheten.",2022-05-13T15:51:49Z,eEn1W9,Varför kapar Ryssland försörjningen av el till Finland?,Vilken procentandel av den totala elförbrukningen i Finland påverkas av Rysslands besked enligt Fingrid?
1999,Pandemidrabbat hotell har anställt ukrainare på flykt,"Siggesta gård drabbades hårt av pandem